In [1]:
from typing import overload
import os

import torch
from torch import Tensor
from torch.utils.data import Dataset, DataLoader, random_split
import config
from torchvision.datasets import MNIST, CIFAR10
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.pyplot as plt


def get_PATH(name):
	'''
	get PATH to store data
	'''
	if config.NUM_LABELLED == -1:
		return f'{config.USED_DATA}/{name}/_full'
	else:
		return f'{config.USED_DATA}/{name}/_{config.NUM_LABELLED}'

def set_random_seed(seed: int) -> None:
    """
    Sets the seeds at a certain value.
    :param seed: the value to be set
    """
    print("Setting seeds ...... \n")
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


class Doodle(Dataset):
	def __init__(self, root: str, train: bool = True, transform = None, target_transform = None, download = False):
		self.root = root + "/Doodles/"

		self.num_classes = len(os.listdir(self.root))

		self.train = train
		if train:
			self.num_data = 100000 * self.num_classes
		else:
			self.num_data = 10000 * self.num_classes

		self.data = np.empty([self.num_data, 28, 28], np.int64)
		self.target = np.empty([self.num_data], dtype=np.int8)

		self.classes = []

		self.download_data()

		self.transform = transform
		self.target_transform = target_transform
	
	def download_data(self):
		for i, file in enumerate(os.listdir(self.root)):
			data: np.ndarray = np.load(self.root + file)
			if self.train:
				data = data[: 100000]
				n = 100000
				data = data.reshape(100000, 28, 28)
			else:
				data = data[100000: 110000]
				n = 10000
				data = data.reshape(10000, 28, 28)
			
			self.data[i*n: (i+1)*n] = data
			
			target = file[18: -4]

			self.target[i*n: (i+1)*n] = np.full([n], i)

			self.classes.append(f'{i} - {target}')




In [46]:
dataset = Doodle("./data", train = False)
print(dataset.classes)

['0 - helicopter', '1 - car', '2 - book', '3 - windmill', '4 - cat', '5 - umbrella', '6 - octopus', '7 - bird', '8 - hat', '9 - birthday cake']
